In [1]:
import sys
sys.path.append('../')
sys.path.append('../Nets/')
from glob import glob
import os
from os.path import join, basename
from multiprocessing import Pool
from scipy.ndimage.interpolation import rotate
from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import SGD, Adam
from tqdm import tqdm
from functools import partial
%pylab inline
from ArchNet import*
from paths import *

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
class LoggingCallback(Callback):
    """Callback that logs message at end of epoch.
    """

    def __init__(self, print_fcn=print):
        Callback.__init__(self)
        self.print_fcn = print_fcn

    def on_epoch_end(self, epoch, logs={}):
        msg = "Epoch: %i, %s" % (epoch, ", ".join("%s: %f" % (k, v) for k, v in logs.iteritems()))
        self.print_fcn(msg)


def print_fcn(msg):
    msg = [msg]
    if isfile(join('/home/a.dobrenkii/', 'archlog_' + POSTFIX)):
        msg += pickle.load(join('/home/a.dobrenkii/', 'archlog_' + POSTFIX))
    pickle.dump(join('/home/a.dobrenkii/', 'archlog_' + POSTFIX), msg)

In [3]:
CPU = 20
ANGLE = 90
POSTFIX = 'a'
NODULE_OVERSAMPLING = 81
SHIFT = 2
SHAPE = (6, 20, 20)
VALID_FOLD = '0'
NB_EPOCH = 50
BATCH_SIZE = 200

In [4]:
archmodel = arch_A()
adam = Adam(lr=.3, decay=1e-5)
archmodel.compile(loss='binary_crossentropy', optimizer=adam)

In [5]:
def cae_augment_patch(patch_mute, shape, shift=2):
    patch, mute = patch_mute
    if mute > 0:
        shift = random.randint(-shift, shift, 3)
        angle = mute % 4 * 90
        if angle != 0:
            patch = rotate(patch, angle, axes=[1,2])
            
    
    center = array(patch.shape) // 2 
    if mute > 0:
        center += shift
        
    left = array(shape) // 2
    right = array(shape) - left
    patch = patch[center[0] - left[0]:center[0] + right[0], 
                  center[1] - left[1]:center[1] + right[1], 
                  center[2] - left[2]:center[2] + right[2]]
    
    return patch

In [6]:
def cae_batch_generator(patch_paths, 
                        batch_size, shape=(6, 20, 20), 
                        angle=15, shift=4, CPU=4):
    
    number_of_batches = ceil(len(patch_paths) / batch_size)
    counter = 0
    
    random.shuffle(patch_paths)
    
    while True:
        batch_files = patch_paths[batch_size * counter:batch_size * (counter + 1)]
        
        shifts = [patch_path.split('_shift')[-1] for patch_path in batch_files] 
        shifts = [int(shift) if shift in ' '.join(arange(NODULE_OVERSAMPLING).astype(str)) else -1
                  for shift in shifts]
        
        batch_files = [patch_path.split('_shift')[0] for patch_path in batch_files] 
        patch_list = [load(patch_path) for patch_path in batch_files]
        
        labels = [[1, 0] if 'NODULE' in patch_path else [0, 1] 
                  for patch_path in batch_files]
        
        augment = partial(cae_augment_patch, shape=shape, shift=shift)
        with Pool(CPU) as pool:
            patch_list = pool.map(augment, list(zip(patch_list, shifts)))
            
        patch_list = array(patch_list)
        patch_list = clip(patch_list, -1000, 400)
        patch_list = (patch_list - patch_list.min()) / (patch_list.max() - patch_list.min())
        
        counter += 1
        
        yield expand_dims(patch_list, 1), asarray(labels)
        
        if counter == number_of_batches:
            random.shuffle(patch_paths)
            counter = 0

In [7]:
valid_fold = '0'
file_list = [path + '_shift' + str(i) 
             for path in glob(join(PATH['LUNA_NODULES_WOR'], 'subset*', '*.npy')) 
             for i in range(NODULE_OVERSAMPLING)]
file_list = set(file_list
             + [path for path in glob(join(PATH['LUNA_VESSELS_WOR'], 'subset*', '*.npy'))])

valid = [path if '_shift' not in path else path.split('_shift')[0] 
         for path in file_list if 'subset' + valid_fold in path]
train = list(file_list.difference(valid))


In [22]:
file_list = [path for path in glob(join(PATH['LUNA_NODULES_WOR'], 'subset*', '*.npy'))] \
          + [path for path in glob(join(PATH['LUNA_VESSELS_WOR'], 'subset*', '*.npy'))]
for file in tqdm(file_list):
    patch = load(file)
    if any(array(patch.shape) != array([64, 64, 64])):
        os.remove(file)

KeyboardInterrupt: 

In [8]:
train_generator = cae_batch_generator(train, 
                                      batch_size=BATCH_SIZE, 
                                      shape=SHAPE, 
                                      shift=SHIFT, 
                                      CPU=CPU)

valid_generator = cae_batch_generator(valid, 
                                      batch_size=BATCH_SIZE, 
                                      shape=SHAPE, 
                                      shift=0, 
                                      CPU=CPU)

In [9]:
checkpoint = ModelCheckpoint(filepath=join(PATH['WEIGHTS'], 'CUMedVis_arch_a'), 
                             verbose=1, 
                             save_best_only=True)

archmodel.fit_generator(train_generator, 
                        samples_per_epoch=len(train), 
                        nb_epoch=NB_EPOCH, 
                        callbacks=[checkpoint], 
                        validation_data=valid_generator, 
                        class_weight=None, 
                        nb_val_samples=len(valid),
                        nb_worker=1)
    
model.save_weights(join(PATH['WEIGHTS'], 'CUMedVis_arch_a_final'))

Epoch 1/50


InternalError: Dst tensor is not initialized.
	 [[Node: Const_35 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [153600,150] values: 0 0 0...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
Caused by op 'Const_35', defined at:
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/a.dobrenkii/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, 